# Agentic AI

code snippets from this section

In [2]:
#%pip install --upgrade --quiet pydantic-ai-slim[anthropic] anthropic

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv("../keys.env")

MODEL_ID="claude-3-7-sonnet-latest"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"

In [2]:
# Needed in Jupyter environment See: https://ai.pydantic.dev/troubleshooting/ 
import nest_asyncio
nest_asyncio.apply()

## PydanticAI

In [4]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    name: str
    quantity_on_hand: int
    weekly_quantity_sold_past_n_weeks: [int]
    weeks_to_deliver: int

@dataclass
class Reorder:
    name: str
    quantity_to_order: int
    reason_to_reorder: str

items = [
    InventoryItem("itemA", 300, [50, 70, 80, 100], 2),
    InventoryItem("itemB", 100, [70, 80, 90, 70], 2),
    InventoryItem("itemC", 200, [80, 70, 90, 80], 1)
]

In [9]:
from pydantic_ai import Agent

agent = Agent(f"anthropic:{MODEL_ID}",
              system_prompt="You are an inventory manager who orders just in time.",
             result_type=list[Reorder])

result = agent.run_sync(f"""
Identify which of these items need to be reordered this week.

**Items**
{items}
""")
print(result.data)

[Reorder(name='itemB', quantity_to_order=300, reason_to_reorder='Current stock (100) is insufficient to cover projected demand over delivery time. Based on recent weekly sales (70-90 units), we need to order enough to cover the 2-week delivery period plus maintain safety stock.'), Reorder(name='itemC', quantity_to_order=100, reason_to_reorder='Current stock (200) is adequate for immediate needs, but with 1-week delivery time and recent weekly sales of 70-90 units, we should place a moderate order to maintain optimal inventory levels.')]
